In [1]:
############ Debugging Required ############

'''
    MNIST digit classification using tensorflow
    dataset: Digit recognizer (Kaggle)
'''

import tensorflow as tf
import pandas as pd

In [2]:
# importing the training set
data = pd.DataFrame(pd.read_csv('train.csv'))
data.sample(frac=1)
data = data.as_matrix()

In [3]:
import numpy as np

# this variable iterates with a step size of batch_size variable
batch_count = 0 
# converting the labels to hot matrix
def hot_matrix(digit_label, size):
    hm = np.zeros([size, 10], dtype = np.float)
    
    for j in range(size):
        hm[j, digit_label[j]] = 1
    return hm

# create batches
def create_batch(X, Y, ep, dataset_size):
    # total batches
    global batch_count
    batch_size = int(dataset_size/ep)
    x_bat = X[batch_count:(batch_count+batch_size), :]
    y_bat = Y[batch_count:(batch_count+batch_size), :]
    return x_bat, y_bat

In [4]:
# data info

# training pixel values
pixel_values = data[0:25000, 1:]

# dataset length
m = np.size(pixel_values[:, 0])
digits = data[0:25000, 0:1]
# labels
labels = hot_matrix(digits, m)

print('pixel_values shape: ', pixel_values.shape)
print('labels shape: ', labels.shape)

pixel_values shape:  (25000, 784)
labels shape:  (25000, 10)


In [5]:
# placeholder for pixel_values
x = tf.placeholder(tf.float32, [None, 784])
# placeholder for labels
y = tf.placeholder(tf.float32, [None, 10])

################## Randomize the weight initialisation ##################


# total layers = 3
# weights
w1 = tf.Variable(tf.random_normal([784, 20]))
w2 = tf.Variable(tf.random_normal([20, 20]))
w3 = tf.Variable(tf.random_normal([20, 10]))

# biases
b1 = tf.Variable(tf.random_normal([20]))
b2 = tf.Variable(tf.random_normal([20]))
b3 = tf.Variable(tf.random_normal([10]))

In [6]:
# hidden layers
z1 = tf.matmul(x, w1) + b1
a1 = tf.nn.relu(z1)
z2 = tf.matmul(a1, w2) + b2
a2 = tf.nn.relu(z2)
z3 = tf.matmul(a2, w3) + b3

# optimization function softmax
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = z3, labels = y) 

# gradient descent
optimize = tf.train.GradientDescentOptimizer(0.05).minimize(softmax_cross_entropy)

In [7]:
# learning starts now
epochs = 1000
# creating Session
s = tf.InteractiveSession()

# initializing variables
tf.global_variables_initializer().run()

for i in range(epochs):
    x_batch, y_batch = create_batch(pixel_values, labels, epochs, m)
    train_step = s.run(optimize, {x: x_batch, y: y_batch})
    cost = s.run(softmax_cross_entropy, {x: x_batch, y: y_batch})
# reset the batch index counter    
global batch_count
batch_count = 0

In [8]:
# test set accuracy
test_data = data[25001:]
# test set size
m_test = np.size(test_data[:, 0])
# test set pixel values
test_pixel_values = test_data[:, 1:]
# creating hot matrix of test set labels
test_labels =  hot_matrix(test_data[:, 0:1], m_test)

# forward propagation
final_softmax = tf.nn.softmax(z3)

prediction = s.run(final_softmax, {x: test_pixel_values, y: test_labels})

max_index = tf.equal(tf.argmax(prediction, 1), tf.argmax(test_labels, 1))
accuracy = tf.reduce_mean(tf.cast(max_index, "float"))

accuracy_output = s.run(accuracy, {x: test_pixel_values, y: test_labels})

print(accuracy_output)

0.0995941
